# Load and prepare the data.


Load your dataset containing stock symbols and dates from a CSV file into a pandas DataFrame.



In [15]:
import os,sys

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

%run BASE_DIR/scripts/eda.py

<Figure size 640x480 with 0 Axes>

In [16]:
data

,headline,publisher,date,stock
10,30 Stocks Moving in Friday's Pre-Market Session,Lisa Levin,2020-05-22,A
11,SVB Leerink Maintains Outperform on Agilent Te...,vishwanath@benzinga.com,2020-05-22,A
12,8 Stocks Moving In Thursday's After-Hours Session,Tyree Gorges,2020-05-21,A
13,Agilent Technologies shares are trading higher...,Benzinga Newsdesk,2020-05-21,A
14,Agilent Technologies Q2 Adj. EPS $0.71 Beats $...,Benzinga Newsdesk,2020-05-21,A
...,...,...,...,...
1407323,Top Narrow Based Indexes For August 29,Monica Gerson,2011-08-29,ZX
1407324,Recap: Wednesday's Top Percentage Gainers and ...,Benjamin Lee,2011-06-22,ZX
1407325,UPDATE: Oppenheimer Color on China Zenix Auto ...,BenzingaStaffL,2011-06-21,ZX
1407326,Oppenheimer Initiates China Zenix At Outperfor...,Joe Young,2011-06-21,ZX


In [17]:
data.columns

Index(['headline', 'publisher', 'date', 'stock'], dtype='object')

In [18]:
data = data.drop(columns=['headline', 'publisher'])

# Determine date ranges and fetch stock data.


Calculate the earliest and latest date for each stock to define the period for which to fetch stock prices.


In [ ]:
import pandas as pd

In [33]:
earliest_dates = data.groupby('stock')['date'].min().reset_index()
earliest_dates.columns = ['stock', 'Earliest']

latest_dates = data.groupby('stock')['date'].max().reset_index()
latest_dates.columns = ['stock', 'Latest']

# Merge the two DataFrames on 'stock'
result_df = pd.merge(earliest_dates, latest_dates, on='stock')

In [34]:
result_df

,stock,Earliest,Latest
0,A,2009-04-29,2020-05-22
1,AA,2009-08-10,2020-05-14
2,AAC,2010-03-25,2019-08-30
3,AAL,2011-05-16,2020-06-01
4,AAMC,2014-02-26,2019-02-27
...,...,...,...
5152,ZTR,2009-08-10,2015-12-30
5153,ZTS,2013-01-17,2020-05-03
5154,ZU,2013-11-14,2015-08-17
5155,ZUMZ,2009-11-16,2020-04-09


Use yfinance to download stock data for each stock based on the calculated date ranges.


In [22]:
result_df.dtypes


stock               object
Earliest    datetime64[ns]
Latest      datetime64[ns]
dtype: object

In [38]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
symbols = tickers['Symbol'].tolist()
result_df = result_df[result_df['stock'].isin(symbols)]
result_df


,stock,Earliest,Latest
0,A,2009-04-29,2020-05-22
3,AAL,2011-05-16,2020-06-01
10,AAPL,2020-03-09,2020-06-02
19,ABBV,2013-01-02,2020-05-14
44,ACGL,2009-11-06,2020-05-20
...,...,...,...
5086,XOM,2018-06-27,2020-06-03
5109,XYL,2011-10-24,2020-05-13
5123,YUM,2011-12-05,2020-05-26
5133,ZBRA,2010-02-09,2020-04-29


In [39]:

def fetch_stock_data(symbol, start_date, end_date):
    try:
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data['stock'] = symbol  # Add a 'Symbol' column
        return stock_data
    except Exception as e:
        print(f"Failed to fetch data for {symbol}: {e}")
        return None

# List to store stock data for each stock symbol
stock_data_list = []

# Fetch stock data for each stock symbol based on the calculated date ranges
for index, row in result_df.iterrows():
    symbol = row['stock']
    start_date = row['Earliest']
    end_date = row['Latest']

    stock_data = fetch_stock_data(symbol, start_date, end_date)

    # Append fetched stock data to the list
    if stock_data is not None:
        stock_data_list.append(stock_data)

# Concatenate stock data from the list into a single DataFrame
stock_data_df = pd.concat(stock_data_list)
stock_data_df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Open,High,Low,Close,Adj Close,Volume,stock
Date,,,,,,,
2009-04-29,12.310443,13.011445,12.238913,12.896996,11.771303,5516508.0,A
2009-04-30,12.947067,13.547926,12.947067,13.061516,11.921465,4977998.0,A
2009-05-01,13.025751,13.590844,13.004292,13.454936,12.280547,4039241.0,A
2009-05-04,13.454936,13.891273,13.412017,13.891273,12.678797,3885042.0,A
2009-05-05,13.848355,13.876967,13.283262,13.397711,12.228311,4895656.0,A
...,...,...,...,...,...,...,...
2020-04-27,126.959999,128.600006,126.099998,127.419998,126.183578,2202200.0,ZTS
2020-04-28,128.460007,128.699997,122.849998,123.000000,121.806480,3098400.0,ZTS
2020-04-29,124.269997,126.980003,123.010002,125.550003,124.331734,2905300.0,ZTS
